In [16]:
from pathlib import Path
from chart_functions import chart2dict

# Import the processed .chart file

# Define path to chart
d = str(Path().resolve().parent)
chartpath = d+'\\tensor-hero\exploder_notes.chart'

notes, song_metadata, time_signatures, BPMs = chart2dict(chartpath)



In [33]:
print(song_metadata)
# print(notes['tick'][0])
# print(notes['N_S'][0])
print(notes.keys())
# print(notes['tick'])

{'Name': '"Exploder"', 'Artist': '"Audioslave"', 'Charter': '"Chezy"', 'Offset': 0, 'Resolution': 192, 'Genre': '"rock"', 'MediaType': '"cd"', 'MusicStream': '"song.ogg"'}
dict_keys(['tick', 'N_S', 'note', 'duration'])


Still need to account for held notes

In [30]:
# Create a dictionary where the keys are the tick values and the values
    # are a list of notes corresponding to the data
coded_notes = {}

# Loop through song one note at a time, processing along the way
for i in range(len(notes['tick'])):
    if notes['N_S'] == 'S':  # if the token is a star power indicator, skip it
        continue

    if notes['tick'][i] not in coded_notes:  # If the key is not in the dictionary
        coded_notes[notes['tick'][i]] = []                   # Create empty list

    if notes['duration'][i] == 0:  # If the note is not held
        coded_notes[notes['tick'][i]].append(int(notes['note'][i]))  # Add note to list
    else:  # If the note has some duration
        if (notes['tick'][i] + notes['duration'][i]) not in coded_notes:  # If the key is not in the dictionary
            coded_notes[notes['tick'][i] + notes['duration'][i]] = []     # Create empty list
        # Add the note with a hold key
        # Hold key is the code of the note + 10 (so 11 indicates release green, 12 is release red, etc.)
        coded_notes[notes['tick'][i]].append(int(notes['note'][i]) + 10)  # Add note to list

        # Add a release key at time step when note will be released.
        # Release key is the code of the note + 20 (so 21 indicates release green, 22 is release red, etc.)
        coded_notes[notes['tick'][i] + notes['duration'][i]].append(int(notes['note'][i]) + 20)  # Add note to list

In [32]:
print(coded_notes[4274])

[12, 5]


## how to read coded_notes
coded_notes is a dictionary with the keys representing ticks and values decoded as follows:

| Value | Interpretation | Value | Interpretation |
| --- | --- | ---|---|
| 0 | green note | 5 | force note flag
| 10 | green note hold | 6 | tap note flag
| 20 | green note release |
| 1 | red note | 
| 11 | red note hold |
| 21 | red note release |
| 2 | yellow note |
| 12 | yellow note hold |
| 22 | yellow note release | 
| 3 | blue note | 
| 13 | blue note hold |
| 23 | blue note release |
| 4 | orange note |
| 14 | orange note hold |
| 24 | orange note release |
| 7 | open note

The flags are always listed after a note value. For example, a 10 followed by a 5 says 'hold a tapped green note.'

## next steps
There are 15 notes and 3 flags (including a flag for regular notes), 15 \* 3 + 1 = 46 possible combinations (including no note). These will be the 'bag of notes.' The next
step is to translate coded_notes into a tensor with the features *timestamp, note event*, with a timestamp existing for every 10ms of the song.

In [31]:
print(coded_notes[4274]

[12, 5]
